In [69]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import balanced_accuracy_score, f1_score, accuracy_score, precision_score, recall_score

In [52]:
###Y-target is severity_class

In [53]:
df = pd.read_csv("final_ohc_wto_sde_train.csv")
y = df["severity_class"].astype("int32").values.ravel()
X = df.drop(columns=["severity_class","damage","ev_id","ev_highest_injury"])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

N_features = X_train.shape[1]
model = Sequential([
    Dense(512, input_shape=(N_features,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(9, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=50,
)

Epoch 1/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6606 - loss: 1.0538 - val_accuracy: 0.6731 - val_loss: 0.9614
Epoch 2/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6661 - loss: 0.9379 - val_accuracy: 0.6715 - val_loss: 0.9435
Epoch 3/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6747 - loss: 0.9239 - val_accuracy: 0.6695 - val_loss: 0.9316
Epoch 4/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6810 - loss: 0.9142 - val_accuracy: 0.6735 - val_loss: 0.9303
Epoch 5/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6778 - loss: 0.9128 - val_accuracy: 0.6675 - val_loss: 0.9342
Epoch 6/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6806 - loss: 0.8996 - val_accuracy: 0.6759 - val_loss: 0.9340
Epoch 7/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6835 - loss: 0.8991 - val_accuracy: 0.6614 - val_loss: 0.9382
Epoch 8/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6812 - loss: 0.8878 - val_accuracy: 0.

In [56]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6430 - loss: 1.2838
Test accuracy: 0.6405


In [57]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [58]:
from sklearn.metrics import precision_score, recall_score
balanced_acc = balanced_accuracy_score(y_test, y_pred_classes)
weighted_f1 = f1_score(y_test, y_pred_classes, average="weighted")
acc = accuracy_score(y_test, y_pred_classes)
weighted_recall = recall_score(y_test, y_pred_classes, average="weighted")
weighted_precision= precision_score(y_test, y_pred_classes, average="weighted")

print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f'Weighted_recall: {weighted_recall:.4f}')
print(f'Weighted_precision: {weighted_precision:.4f}')

Balanced Accuracy: 0.1435
Weighted F1 Score: 0.5690
Accuracy: 0.6405
Weighted_recall: 0.6405
Weighted_precision: 0.5316


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [59]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [60]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["severity_class"]

In [92]:
y_h_pred = model.predict(x_h)
y_h_pred= np.argmax(y_h_pred, axis=1) 

98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [65]:
from utils import utils
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.64
Balanced Accuracy: 0.14
Precision: 0.53
Recall: 0.64
F1 Score: 0.56


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
###y-target is damage

In [66]:
df = pd.read_csv("final_ohc_wto_sde_train.csv")
y = df["damage"].astype("int32").values.ravel()
X = df.drop(columns=["severity_class","damage","ev_id","ev_highest_injury"])

In [67]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

N_features = X_train.shape[1]
model2 = Sequential([
    Dense(512, input_shape=(N_features,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(9, activation="softmax")
])

model2.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [68]:
history = model2.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=50,
)

Epoch 1/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8742 - loss: 0.4587 - val_accuracy: 0.8947 - val_loss: 0.2849
Epoch 2/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8885 - loss: 0.3106 - val_accuracy: 0.8943 - val_loss: 0.2805
Epoch 3/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8917 - loss: 0.3074 - val_accuracy: 0.8947 - val_loss: 0.2834
Epoch 4/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8927 - loss: 0.2934 - val_accuracy: 0.8906 - val_loss: 0.2917
Epoch 5/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8941 - loss: 0.2910 - val_accuracy: 0.8943 - val_loss: 0.2785
Epoch 6/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9021 - loss: 0.2816 - val_accuracy: 0.8934 - val_loss: 0.2788
Epoch 7/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8934 - loss: 0.2860 - val_accuracy: 0.8926 - val_loss: 0.2784
Epoch 8/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8949 - loss: 0.2794 - val_accuracy: 0.

In [35]:
y_pred = model2.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
balanced_acc = balanced_accuracy_score(y_test, y_pred_classes)
weighted_f1 = f1_score(y_test, y_pred_classes, average="weighted")
acc = accuracy_score(y_test, y_pred_classes)
weighted_recall = recall_score(y_test, y_pred_classes, average="weighted")
weighted_precision= precision_score(y_test, y_pred_classes, average="weighted")

print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f'Weighted_recall: {weighted_recall:.4f}')
print(f'Weighted_precision: {weighted_precision:.4f}')

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Balanced Accuracy: 0.4565
Weighted F1 Score: 0.8780
Accuracy: 0.8979
Weighted_recall: 0.8979
Weighted_precision: 0.8755


In [90]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["damage"]
y_h_pred = model2.predict(x_h)
y_h_pred=np.argmax(y_h_pred, axis=1)

98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [91]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.88
Balanced Accuracy: 0.41
Precision: 0.85
Recall: 0.88
F1 Score: 0.86


In [ ]:
###y-target is ev_highest_injury

In [70]:
df = pd.read_csv("final_ohc_wto_sde_train.csv")
y = df["ev_highest_injury"].astype("int32").values.ravel()
X = df.drop(columns=["severity_class","damage","ev_id","ev_highest_injury"])

In [71]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

N_features = X_train.shape[1]
model3 = Sequential([
    Dense(512, input_shape=(N_features,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(9, activation="softmax")
])

model3.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [72]:
history = model3.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=50,
)

Epoch 1/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7585 - loss: 0.7627 - val_accuracy: 0.7708 - val_loss: 0.6513
Epoch 2/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7658 - loss: 0.6505 - val_accuracy: 0.7704 - val_loss: 0.6443
Epoch 3/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7719 - loss: 0.6407 - val_accuracy: 0.7708 - val_loss: 0.6438
Epoch 4/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7714 - loss: 0.6277 - val_accuracy: 0.7708 - val_loss: 0.6398
Epoch 5/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7723 - loss: 0.6294 - val_accuracy: 0.7700 - val_loss: 0.6383
Epoch 6/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7758 - loss: 0.6154 - val_accuracy: 0.7700 - val_loss: 0.6488
Epoch 7/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7735 - loss: 0.6206 - val_accuracy: 0.7708 - val_loss: 0.6398
Epoch 8/50
311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7719 - loss: 0.6201 - val_accuracy: 0.

In [73]:
y_pred = model2.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
a, b, c, d, e = utils.print_metrics(y_test, y_pred)

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.72
Balanced Accuracy: 0.32
Precision: 0.61
Recall: 0.72
F1 Score: 0.65


In [81]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["ev_highest_injury"]

In [85]:
y_h_pred=model2.predict(x_h)
y_h_pred = np.argmax(y_h_pred, axis=1)
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.72
Balanced Accuracy: 0.32
Precision: 0.60
Recall: 0.72
F1 Score: 0.65
